In this session, we are going to make practical use of what we just learned with using high performance computing to explore a region of the genome, the RHD locus, which is responsible for the Rh antigen that can have an adverse effect during some pregnancies. This gene is located at chr1:25598977-25656936 in the hg19 version of the human genome reference sequence. We are going to use Python to help manage our data as it is well suited for this.

We are also going to make use of some genomics software packages and data formats that were just introduced, specifically *samtools* and *bam format*

We need to first set imported packages and specific file locations

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import re
from pylab import plot,show
from scipy import stats
import os

#update to your own folder
baseDir='/scratch/biobootcamp_fluxod/remills/day4/read_counts_by_region'
os.chdir(baseDir)

#take the baseDir and concatentate on individual file names
fnSampleInfo = '%s/%s'%( baseDir, 'sample_info.txt' )
fnSampleList = '%s/%s'%( baseDir, 'sample_names.txt' )

#full path to samtools package
samtools = '/sw/med/centos7/samtools/0.1.19/samtools'

#define some empty global dictionaries
samplePath={}
samplePop={}
    
#define region of interest
chromo = "1"
startPos = 25598977
endPos = 25656936
region = "1:25598977-25656936" #format needed for calling samtools

Most of these packages should be familiar from Tuesday (e.g. matplotlib, numpy, scipi), but a few may be new. 
- *re* is a package that allows the use of regular expressions
- *os* allows for the use of operating specific commands (e.g mkdir)
- *subprocess* allows us to construct more complicated system calls, which we will need when creating our samtools command line

Let's take a quick look at the sample info file to check it's format.

In [ ]:
fileSampleInfo = open(fnSampleInfo,'r')
line = fileSampleInfo.readline()
fileSampleInfo.close()

line = line.rstrip()
line = line.split('\t')
line

This file appears to contain various metadata for a number of individual. We are particularly interested in two fields, namely *Sample* which gives a unique identifier for an individual, and *File Path* which gives an absolute location to the alignment file for each respective sample. To facilitate the input of this information into a data structure we can make use of, let's define a function to read through the file line by line and store the data for each sample and its file location into a dictionary

In [ ]:
def parseSampleTable( filename ):
    fileSampTable = open(filename,'r')
    
    # skip past the file header
    fileSampTable.readline()

    #define dictionaries to store the data we read in
    samplePath={}
    samplePop={}
    
    # go line by line
    for line in fileSampTable:
        #first strip off the *end of line* character with rstrip
        #then split the line into a list, using the tab character '\t' as delimiter
        line = line.rstrip().split('\t')
        
        #since we know what the indices are for our variables of interest, we can access them directly
        samplePath[ line[0] ] = line[5]
        samplePop[ line[0] ] = line[2]
    
    #return our two dictionaries for use elsewhere in the notebook
    return samplePath, samplePop

We can now call this function using the file name variable we defined above and assign it to our previously defined global dictionaries

In [ ]:
samplePath, samplePop = parseSampleTable( fnSampleInfo )

Let's test it out with one of our sample names

In [ ]:
#example
sample = "HG00112"
samplePath[sample]

Our goal now is to count the sequence reads at the RHD locus from each file and compare them.  We can do this using the *samtools* command we talked about earlier along with the unix command *wc -l* which will count the number of lines printed to thes screen. We can call this program from within python using the *subprocess.call()* function

As a review, *samtools* has the following usage:
*samtools view [options] <in.bam>|<in.sam> [region1 [...]]*

So, we need to give it a path to the in.bam file and the region we are interested in. Since *samtools* is a standalone software package, we can construct our command line in Python and run it as though we were using the command line directly using *subprocess.call*

In [ ]:
#construct command as if we would be running it from the command line. 
command = samtools + " view " + samplePath[sample] + " " + region + " | wc -l > " + sample + ".cnt"
print(command)
subprocess.call(command, shell=True)

The output of *0* indicates that there were no errors returned when running this system call

With this command, we:
- Pulled out the sequence reads at position 1:25598977-25656936 from the bam file of interest
- *piped* the results to the *wc -l* command, which returns the number of lines
- redirected the output to a new file using *>*

Please take a moment to write some code in the block below to open this file in Jupyter and look at it's output (*hint*, you can reuse some code we made earlier in the notebook!)

Each individual genome may be sequenced to different depths of coverage, leading to differences in read counts being a reflection of that depth rather than a true difference in sequence content. Thus, it would make sense to normalize by the total number of reads generated either for the whole genome or, as we will do here, for the chromosome of interest.

In [ ]:
#Only need data for chromosome 1, so 'head -1' returns only the first line of the output that corresponds to chromo 1
#Number of aligned reads is in column 3, so we can 'cut' this column out and be left with only a single data point
command = samtools + " idxstats " + samplePath[sample] + " | head -1 | cut -f 3 > " + sample + ".num"
print(command)
subprocess.call(command, shell=True)

However, with over 2500 sequences it would be inefficient (and boring!) to do all of them manually in this fashion. We could make use of cluster computing to run all 2500 simultaneously, however this would result in 2500 individual jobs that would need to be submitted, managed, and reviewed.

Instead, we can *batch* samples into smaller collections and send each batch as its own job. To do this, we will need to determine both **(a)** the number of batches we wish to submit and **(b)** the number of samples to include in each batch. For this exercise, we will construct batches of n=50.

However, with over 2500 sequences it would be inefficient (and boring!) to do all of them manually in this fashion. We could make use of cluster computing to run all 2500 simultaneously, however this would result in 2500 individual jobs that would need to be submitted, managed, and reviewed.

Instead, we can *batch* samples into smaller collections and send each batch as its own job. We can do this by creating a series of batch files containing the commands we wish to run, and then submit them as a *job array* to the cluster.

Let's iterate through our samplePath dictionary and construct our commands for each as above, writing them as needed to their respective batch files

In [ ]:
#set target number of batches (jobs)
numBatches = 50

#length of dictionary = number of samples here, can be calculated with len(dict)
numSamplesPerBatch = int(len(samplePath) / numBatches) + 1; #add one to round up if needed

#show summary
print(len(samplePath))
print(numSamplesPerBatch)

#keep track of what the last sample number was
lastSampleNum = 0;
#create a list of sample ids, to match the sample paths
samples = list(samplePath.keys())

#create a directory to store all our batch, log, and output files
if not os.path.exists(baseDir + "/batches"):
    os.makedirs(baseDir + "/batches") 

#create a directory to store stdout and stderr files
if not os.path.exists(baseDir + "/logs"):
    os.makedirs(baseDir + "/logs") 
    
if not os.path.exists(baseDir + "/counts"):
    os.makedirs(baseDir + "/counts")

    
for batchNum in range (numBatches):
    #open a command file for this batch
    fileBatch = open(baseDir+"/batches/batch"+str(batchNum)+".sh", "w")
    
    #set the range for this batch
    startIndex = lastSampleNum
    endIndex = lastSampleNum + numSamplesPerBatch
    
    #make sure last index is in range, in case batches aren't divided evenly
    if endIndex > len(samplePath): 
        endIndex = len(samplePath)
        
    for sampleNum in range (startIndex, endIndex):
        sample = samples[sampleNum] #current sample
        #write command for region counts
        command = samtools + " view " + samplePath[sample] + " " + region + " | wc -l > " + baseDir + "/counts/" + sample + ".cnt"
        fileBatch.write(command+"\n")
        #write command for chromosome counts
        command = samtools + " idxstats " + samplePath[sample] + " | head -1 | cut -f 3 > " + baseDir + "/counts/" + sample + ".num"
        fileBatch.write(command+"\n")
    fileBatch.close()
    
    #update index for next iteration
    lastSampleNum = lastSampleNum + numSamplesPerBatch

Then, we can use python to create a parent PBS file that we can use for all batch files and subsequently submit them to the cluster.

In [ ]:
#create new PBS file for submission, with corresponding header
filePBS = open(baseDir+"/day4.pbs", "w")
filePBS.write("#!/bin/bash\n") #shell line
filePBS.write("#\n") #spacer
filePBS.write("#PBS -N bioboot_day4\n") #name of job
filePBS.write("#PBS -t 0-49\n") #set the number of batches to run
filePBS.write("#PBS -o " + baseDir + "/logs/batch.stdout\n") #stdout file
filePBS.write("#PBS -e " + baseDir + "/logs/batch.stderr\n") #stderr file
filePBS.write("#PBS -l procs=1,qos=flux,mem=4gb,walltime=24:00:00\n") #feature line
filePBS.write("#PBS -m a\n") #message line, only send errors to email
filePBS.write("#PBS -M remills@umich.edu\n") #email line, to send messages from above
filePBS.write("#PBS -A biobootcamp_fluxod\n") #which account to associate job
filePBS.write("#PBS -q fluxod\n") #which queue to send job to 
filePBS.write("#PBS -V\n") #pass environmental variables to job
filePBS.write("#PBS -d .\n") #use current working directory
filePBS.write("\n") #spacer

filePBS.write("bash " + baseDir + "/batches/batch${PBS_ARRAYID}.sh\n")
filePBS.close()

Now, let's submit our job array!

In [ ]:
command = "qsub day4.pbs"
print(command)
subprocess.call(command, shell=True)

Now we must wait for the jobs to finish. There are python modules which will check and sleep until all jobs are finalized. For simplicity, we can just manually check with the *qstat* command you were introduced to earlier (use your unique name instead of mine!).

In [ ]:
!qstat -u remills

We can view all jobs in the array as well

In [ ]:
!qstat -t 24299940[]

Or the status of an individual job itself

In [ ]:
!qstat -t 24299940[1]

This might take a few minutes to run. Once completed, we should take the time to do some quick checks of the stderr files to make sure that there were not  issues with any particular batch. This can be as simple as a quick *cat* of all the stderr files to see if anything is there. The lack of errors sent to your email address is also a sign that things went well.

In [ ]:
!cat {baseDir}/logs/*stderr*

Now, we can use python to read back in all the output files and construct a normalized list of sequence counts

In [ ]:
samples = samplePath.keys()
rhdList = []
for sample in samples:
    #rhd counts
    cntFile = open(baseDir + "/counts/" + sample + ".cnt", "r")
    cnt = cntFile.readline()
    cnt = float(cnt.rstrip()) #float is important, otherwise will treat as STR or INT
    cntFile.close()
    
    #chromo 1 counts
    numFile = open(baseDir + "/counts/" + sample + ".num", "r")
    num = numFile.readline()
    num = float(num.rstrip())
    numFile.close()
    
    normCnt = 1e3 * (cnt / num) #scale for plotting
    rhdList.append(normCnt)
    print(rhdList[-1])


In [ ]:
plt.hist(rhdList, bins=100)
plt.title("RHD Locus Read Counts")
plt.xlabel("Sequence Read Counts")
plt.ylabel("Frequency")
plt.show()

As you can see, our samples seem to stratify into three distinct groups based on their depth of sequence coverage at the RHD locus, which represent individuals with 2 (normal) copies of the region, 1 copy, and 0 copies. The last peak is much smaller, and is consistent with observed frequencies of Rh- individuals (~10-15%)